In [1]:
import nltk
import time
import numpy as np
import pandas as pd
from nltk import pos_tag
from sklearn.svm import SVC
from string import punctuation 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import NaiveBayesClassifier as NBC
from nltk.stem import WordNetLemmatizer as WNL
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
time1 = time.time()
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")
data_train.head(3)

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)


In [3]:
data_test.head(3)

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)


In [4]:
document_train = data_train["text"]
document_test = data_test["text"]

In [5]:
all_doc_train = []
for i in range(len(data_train)):
    all_doc_train.append((word_tokenize(document_train[i]), data_train["airline_sentiment"][i]))
all_doc_train[:5]

[(['@',
   'SouthwestAir',
   'I',
   'am',
   'scheduled',
   'for',
   'the',
   'morning',
   ',',
   '2',
   'days',
   'after',
   'the',
   'fact',
   ',',
   'yes',
   '..',
   'not',
   'sure',
   'why',
   'my',
   'evening',
   'flight',
   'was',
   'the',
   'only',
   'one',
   'Cancelled',
   'Flightled'],
  'negative'),
 (['@',
   'SouthwestAir',
   'seeing',
   'your',
   'workers',
   'time',
   'in',
   'and',
   'time',
   'out',
   'going',
   'above',
   'and',
   'beyond',
   'is',
   'why',
   'I',
   'love',
   'flying',
   'with',
   'you',
   'guys',
   '.',
   'Thank',
   'you',
   '!'],
  'positive'),
 (['@',
   'united',
   'Flew',
   'ORD',
   'to',
   'Miami',
   'and',
   'back',
   'and',
   'had',
   'great',
   'crew',
   ',',
   'service',
   'on',
   'both',
   'legs',
   '.',
   'THANKS'],
  'positive'),
 (['@',
   'SouthwestAir',
   '@',
   'dultch97',
   'that',
   "'s",
   'horse',
   'radish',
   '😤🐴'],
  'negative'),
 (['@',
   'united',
   's

In [6]:
all_doc_test = []
for i in range(len(data_test)):
    all_doc_test.append(word_tokenize(document_test[i]))
all_doc_test[:5]

[['@',
  'AmericanAir',
  'In',
  'car',
  'gng',
  'to',
  'DFW',
  '.',
  'Pulled',
  'over',
  '1hr',
  'ago',
  '-',
  'very',
  'icy',
  'roads',
  '.',
  'On-hold',
  'with',
  'AA',
  'since',
  '1hr',
  '.',
  'Ca',
  "n't",
  'reach',
  'arpt',
  'for',
  'AA2450',
  '.',
  'Wat',
  '2',
  'do',
  '?'],
 ['@',
  'AmericanAir',
  'after',
  'all',
  ',',
  'the',
  'plane',
  'didn',
  '’',
  't',
  'land',
  'in',
  'identical',
  'or',
  'worse',
  ')',
  'conditions',
  'at',
  'GRK',
  'according',
  'to',
  'METARs',
  '.'],
 ['@',
  'SouthwestAir',
  'ca',
  "n't",
  'believe',
  'how',
  'many',
  'paying',
  'customers',
  'you',
  'left',
  'high',
  'and',
  'dry',
  'with',
  'no',
  'reason',
  'for',
  'flight',
  'Cancelled',
  'Flightlations',
  'Monday',
  'out',
  'of',
  'BDL',
  '!',
  'Wow',
  '.'],
 ['@',
  'USAirways',
  'I',
  'can',
  'legitimately',
  'say',
  'that',
  'I',
  'would',
  'have',
  'rather',
  'driven',
  'cross',
  'country',
  'than',


In [7]:
stop_words = stopwords.words("english")
punc = list(punctuation)
stop_words+=punc
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [8]:
lemm = WNL()

In [9]:
def get_simple_pos(tag): #since pos tag are comlicated make it simpler
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        wordnet.NOUN

In [10]:
def cleaning(text):
    output_words = []
    for word in text:
        if word.lower() not in stop_words and len(word)>2:
            ps = pos_tag(word)
            wn = str(get_simple_pos(ps[0][1]))
            clean_word = lemm.lemmatize(word, pos = wordnet.NOUN)
            output_words.append(clean_word)
    return output_words

In [11]:
start = time.time()
final_doc_train = [(cleaning(text), cat) for text, cat in all_doc_train]
final_doc_test = [cleaning(text) for text in all_doc_test]
print(final_doc_train[:2])
print("---------------------------------------------")
print(final_doc_test[:2])

[(['SouthwestAir', 'scheduled', 'morning', 'day', 'fact', 'yes', 'sure', 'evening', 'flight', 'one', 'Cancelled', 'Flightled'], 'negative'), (['SouthwestAir', 'seeing', 'worker', 'time', 'time', 'going', 'beyond', 'love', 'flying', 'guy', 'Thank'], 'positive')]
---------------------------------------------
[['AmericanAir', 'car', 'gng', 'DFW', 'Pulled', '1hr', 'ago', 'icy', 'road', 'On-hold', 'since', '1hr', "n't", 'reach', 'arpt', 'AA2450', 'Wat'], ['AmericanAir', 'plane', 'land', 'identical', 'worse', 'condition', 'GRK', 'according', 'METARs']]


In [12]:
print(time.time() - start)

503.1250216960907


In [13]:
x_train = []
for i in range(len(final_doc_train)):
    x_train.append(final_doc_train[i][0])
x_train[:2]

[['SouthwestAir',
  'scheduled',
  'morning',
  'day',
  'fact',
  'yes',
  'sure',
  'evening',
  'flight',
  'one',
  'Cancelled',
  'Flightled'],
 ['SouthwestAir',
  'seeing',
  'worker',
  'time',
  'time',
  'going',
  'beyond',
  'love',
  'flying',
  'guy',
  'Thank']]

In [14]:
all_words = []
for i in x_train:
    for j in i:
        all_words.append(j)
all_words

['SouthwestAir',
 'scheduled',
 'morning',
 'day',
 'fact',
 'yes',
 'sure',
 'evening',
 'flight',
 'one',
 'Cancelled',
 'Flightled',
 'SouthwestAir',
 'seeing',
 'worker',
 'time',
 'time',
 'going',
 'beyond',
 'love',
 'flying',
 'guy',
 'Thank',
 'united',
 'Flew',
 'ORD',
 'Miami',
 'back',
 'great',
 'crew',
 'service',
 'leg',
 'THANKS',
 'SouthwestAir',
 'dultch97',
 'horse',
 'radish',
 'united',
 'flight',
 'ORD',
 'delayed',
 'Air',
 'Force',
 'One',
 'last',
 'flight',
 'SBN',
 '8:20',
 'min',
 'landed',
 'united',
 'load',
 'flying',
 'sardine',
 'knew',
 'pilot',
 'hour',
 'Late',
 'Flight',
 'incompetent',
 'beyond',
 'belief',
 'JetBlue',
 'stock',
 'response',
 'Delays',
 'frustrating',
 'poor',
 'cust',
 'serv',
 'amp',
 'told',
 'ppl',
 'wait',
 'amp',
 'come',
 'back',
 'JetBlue',
 'nice',
 'Hoping',
 'rack',
 'enough',
 'mile',
 'take',
 'trip',
 'Seattle',
 'enjoy',
 'perfect',
 'latte',
 'city',
 'coffee',
 'united',
 'frankly',
 'worse',
 'customer',
 'service

In [15]:
freq = nltk.FreqDist(all_words)
features_ = [i for i in freq.most_common(5000)]
feature = []
for i in features_:
    feature.append(i[0])
feature[:15]

['united',
 'flight',
 'AmericanAir',
 'USAirways',
 'SouthwestAir',
 'JetBlue',
 "n't",
 'get',
 'http',
 'hour',
 'Cancelled',
 'service',
 'time',
 'customer',
 '...']

In [16]:
x_train

[['SouthwestAir',
  'scheduled',
  'morning',
  'day',
  'fact',
  'yes',
  'sure',
  'evening',
  'flight',
  'one',
  'Cancelled',
  'Flightled'],
 ['SouthwestAir',
  'seeing',
  'worker',
  'time',
  'time',
  'going',
  'beyond',
  'love',
  'flying',
  'guy',
  'Thank'],
 ['united',
  'Flew',
  'ORD',
  'Miami',
  'back',
  'great',
  'crew',
  'service',
  'leg',
  'THANKS'],
 ['SouthwestAir', 'dultch97', 'horse', 'radish'],
 ['united',
  'flight',
  'ORD',
  'delayed',
  'Air',
  'Force',
  'One',
  'last',
  'flight',
  'SBN',
  '8:20',
  'min',
  'landed'],
 ['united',
  'load',
  'flying',
  'sardine',
  'knew',
  'pilot',
  'hour',
  'Late',
  'Flight',
  'incompetent',
  'beyond',
  'belief'],
 ['JetBlue',
  'stock',
  'response',
  'Delays',
  'frustrating',
  'poor',
  'cust',
  'serv',
  'amp',
  'told',
  'ppl',
  'wait',
  'amp',
  'come',
  'back'],
 ['JetBlue',
  'nice',
  'Hoping',
  'rack',
  'enough',
  'mile',
  'take',
  'trip',
  'Seattle',
  'enjoy',
  'perfec

In [17]:
cv = CountVectorizer(max_features = 10000, ngram_range = (1, 3), df_max = .5)
x_ = cv.fit_transform(all_words)
len(cv.get_feature_names())

10000

In [18]:
def conver_array(arr):
    arr_feature = np.zeros((len(feature)))
    for i in range(len(arr)):
        for j in range(len(feature)):
            if arr[i] == feature[j]:
                arr_feature[j] += 1
    return arr_feature

In [19]:
x_train_array = [conver_array(arr) for arr in x_train]
x_train_array = np.array(x_train_array)
x_train_array.shape

(10980, 5000)

In [20]:
y_train = np.array(data_train["airline_sentiment"])
y_train.shape

(10980,)

In [21]:
final_doc_test

[['AmericanAir',
  'car',
  'gng',
  'DFW',
  'Pulled',
  '1hr',
  'ago',
  'icy',
  'road',
  'On-hold',
  'since',
  '1hr',
  "n't",
  'reach',
  'arpt',
  'AA2450',
  'Wat'],
 ['AmericanAir',
  'plane',
  'land',
  'identical',
  'worse',
  'condition',
  'GRK',
  'according',
  'METARs'],
 ['SouthwestAir',
  "n't",
  'believe',
  'many',
  'paying',
  'customer',
  'left',
  'high',
  'dry',
  'reason',
  'flight',
  'Cancelled',
  'Flightlations',
  'Monday',
  'BDL',
  'Wow'],
 ['USAirways',
  'legitimately',
  'say',
  'would',
  'rather',
  'driven',
  'cross',
  'country',
  'flown',
  'Airways'],
 ['AmericanAir', 'still', 'response', 'great', 'job', 'guy'],
 ['united',
  'developer',
  'flying',
  'tmrw',
  'morn',
  'w/45',
  'min',
  'layover',
  'earlier',
  'flight',
  '1.5hr',
  'layover',
  'move'],
 ['USAirways', 'hello', 'Anyone'],
 ['USAirways',
  'husainhaqqani',
  'Mr.',
  'Husain',
  'shld',
  'protest',
  'well',
  'one',
  'party',
  'member',
  'Rehman',
  'Mal

In [22]:
x_test_array = [conver_array(arr) for arr in final_doc_test]
x_test_array = np.array(x_test_array)
x_test_array.shape

(3660, 5000)

In [23]:
rfc = RFC()
rfc.fit(x_train_array, y_train)
rfc.score(x_train_array, y_train) #0.9934426229508196

0.9934426229508196

In [24]:
y_pred1 = rfc.predict(x_test_array)

In [25]:
df = pd.DataFrame(y_pred1)
df.head()
df.to_csv("y_predictions.csv", header = False, index = False)

In [26]:
print(time.time()- time1)

875.8419620990753
